In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Using TensorFlow backend.
C:\Python\WPy-3670\python-3.6.7.amd64\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Python\WPy-3670\python-3.6.7.amd64\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Python\WPy-3670\python-3.6.7.amd64\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Python\WPy-3670\python-3.6.7.amd64\lib

In [3]:
vbc = pd.read_csv("../data/vb_data_3_categZone.csv")
vb = pd.read_csv("../data/vb_data_3_numZone.csv")
print(len(vb), 'lines loaded')

146050 lines loaded


In [4]:
X = vb.drop(['Season', 'GameID', 'PlayerTeam', 'PlayerName', 'RewardDistance', 'RewardValue'], axis=1)
cols = [col for col in list(X.columns) if X[col].dtype == 'object']
X = pd.get_dummies(data=X, columns = cols)
Y = vb.RewardValue
print(len(X.columns), 'columns in dataframe')

204 columns in dataframe


In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_n = scaler.transform(X)

In [6]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(160, input_dim=204, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, input_dim=160, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal', activation='tanh'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=15, batch_size=64, verbose=0)
kfold = KFold(n_splits=5)

In [10]:
import time
t = time.time()
for k in range(3,20):
    estimator = KerasRegressor(build_fn=baseline_model, epochs=k, batch_size=64, verbose=0)
    results = cross_val_score(estimator, X, Y, cv=kfold)
    print("Result for %d epochs: %.4f (%.2f) MSE" % (k, results.mean(), results.std()))
    #print(results)
    print("Time elapsed:", time.time()-t)

Result for 3 epochs: -0.6233 (0.01) MSE
Time elapsed: 68.32886552810669
Result for 4 epochs: -0.6174 (0.01) MSE
Time elapsed: 171.3441665172577
Result for 5 epochs: -0.6166 (0.01) MSE
Time elapsed: 302.524534702301
Result for 6 epochs: -0.6157 (0.01) MSE
Time elapsed: 474.5486605167389
Result for 7 epochs: -0.6153 (0.01) MSE
Time elapsed: 688.9392251968384
Result for 8 epochs: -0.6180 (0.01) MSE
Time elapsed: 946.8766977787018
Result for 9 epochs: -0.6179 (0.01) MSE
Time elapsed: 1235.9549872875214
Result for 10 epochs: -0.6170 (0.01) MSE
Time elapsed: 1578.1987149715424
Result for 11 epochs: -0.6180 (0.01) MSE
Time elapsed: 1981.4478669166565
Result for 12 epochs: -0.6179 (0.01) MSE
Time elapsed: 2457.8616428375244
Result for 13 epochs: -0.6184 (0.01) MSE
Time elapsed: 3016.3553421497345
Result for 14 epochs: -0.6204 (0.01) MSE
Time elapsed: 3657.088270187378
Result for 15 epochs: -0.6190 (0.01) MSE
Time elapsed: 4401.62668132782
Result for 16 epochs: -0.6200 (0.01) MSE
Time elapsed: 

In [12]:
from sklearn import metrics
for k in range(3,20):
    model = baseline_model()
    model.fit(X_n,Y,epochs=k, shuffle=True, batch_size=64, verbose=0)
    yp = model.predict(X_n)
    print(k, metrics.mean_squared_error(Y,yp))

3 0.614845633050413
4 0.6090885249445158
5 0.6098673522528847
6 0.6087576471036178
7 0.6078304307774619
8 0.606563356283146
9 0.6039002186094222
10 0.6017533567241319
11 0.6016242715165601
12 0.6069657611780048
13 0.604716299466489
14 0.5994722546370327
15 0.6001953628257938
16 0.5993999326150183
17 0.5983801387013684
18 0.5979715852133953
19 0.5957619439576445


In [7]:
estimator.fit(X_n,Y,epochs=15,verbose=1,shuffle=True)



Epoch 1/15
146050/146050 [==============================] - 4s 30us/step - loss: 0.6602: 0s - loss
Epoch 2/15
146050/146050 [==============================] - 4s 28us/step - loss: 0.6177
Epoch 3/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6143
Epoch 4/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6129
Epoch 5/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6112
Epoch 6/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6107
Epoch 7/15
146050/146050 [==============================] - 4s 28us/step - loss: 0.6090
Epoch 8/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6083
Epoch 9/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6073
Epoch 10/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6066
Epoch 11/15
146050/146050 [==============================] - 4s 27us/step - loss: 0.6056
Epoch 12/15
14605

In [8]:
from sklearn import metrics
yp = estimator.predict(X_n)
print(metrics.mean_squared_error(Y,yp))

0.5986944616286725


In [9]:
import pickle
pickle.dump(yp, open('nn_regr.pkl', 'wb'))